<a href="https://colab.research.google.com/github/rjcommand/annotation-python-tools/blob/main/notebooks/Run%20Video%20Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Detection and Tracking Inference
#### Peyton Lee and Neha Nagvekar, 5/31/22

Runs inference on a provided video using a trained YOLOv5 model and the Norfair tracking algorithm.

## Setup
You'll need to download and install the Deepsea-Detector project, which you can find on [GitHub](https://github.com/ShrimpCryptid/deepsea-detector)!

In [1]:
!git clone https://github.com/ShrimpCryptid/deepsea-detector.git

Cloning into 'deepsea-detector'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 432 (delta 24), reused 19 (delta 6), pack-reused 378 (from 1)
Receiving objects: 100% (432/432), 38.74 MiB | 8.58 MiB/s, done.
Resolving deltas: 100% (226/226), done.
Updating files: 100% (74/74), done.


In [2]:
# Update the Deepsea-Detector project if already downloaded!
!cd deepsea-detector; git pull

Already up to date.


In [3]:
# This project uses Git large file system (LFS), so we need to install it and
# fetch our large project files with it.
!cd deepsea-detector; git lfs install; git lfs fetch; git lfs pull

Updated git hooks.
Git LFS initialized.
fetch: Fetching reference refs/heads/main


In [4]:
# Install Deepsea-Detector's dependencies.
!pip install -r deepsea-detector/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Project Setup
You'll need to do the following steps before processing a video.
1. Import your model weights, which should end in `.pt`. You can use the default model included with Deepsea-Detector, or upload your own.
2. Import the video file(s) you want to analyze.

For example, you can download your model weights and video files from an external source using the `curl` command, or upload files to the environment.

If you're using Google Colab, you can upload videos using the file browser on the left pane. *(Be warned that downloading/uploading files from Colab can be rather slow.)*

In [5]:
# Download an example video via URL and save it as `video.mp4`.
video_url = "https://data.nodc.noaa.gov/oer/video/EX1708/Video/EX1708_DIVE01_20170907/Compressed/EX1708_VID_20170907T204000Z_ROVHD_Low.mp4"
!curl -L {video_url} --output video.mp4

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   344    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 56.9M  100 56.9M    0     0  17.5M      0  0:00:03  0:00:03 --:--:-- 26.4M


You can also mount a Google Drive folder to access files, which is much faster than directly uploading/downloading them from Google Colab. You'll get prompted to authorize Google Colab to make changes to your Drive.

In [6]:
# Import and mount your Google Drive folder as a directory that Colab can access.
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [10]:
import torch
# check if cuda is available.
device="cpu"
if not torch.cuda.is_available():
    print("Warning: CUDA is not available. The script will be slower than with CUDA. To enable cuda, go to Runtime -> Change runtime type -> Hardware accelerator -> GPU")
else:
    print("CUDA is available.")
    device="cuda"

CUDA is available.


### Processing a Video
The following script runs the inference for a single video.

In [18]:
# Add apostrophes (') on either side of the path if there are spaces in any of your path names.
MODEL_PATH = "/content/deepsea-detector/models/deepsea-detector.pt"
VIDEO_INPUT_PATH = "/content/deepsea-detector/data/test_input_video/coral_trimmed.mp4"
VIDEO_OUTPUT_PATH = "single_out.mp4"
CSV_OUTPUT_PATH = "single_out.csv"

!python3 deepsea-detector/src/detection.py {VIDEO_INPUT_PATH} \
--detector_path {MODEL_PATH} \
--img_size 640 \
--conf_thres 0.10 \
--period 3 \
--device cpu \
--output_video {VIDEO_OUTPUT_PATH} \
--output_csv {CSV_OUTPUT_PATH}

c ... p4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   1% -:--:-- 137.62fps/usr/local/lib/python3.10/dist-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c ... p4 ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   3% 0:00:57 2.73fps/usr/local/lib/python3.10/dist-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c ... p4 ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   6% 0:00:55 2.39fps/usr/local/lib/python3.10/dist-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c ... p4 ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   6% 0:00:55 2.39fps
Traceback (most recent call last):
  File "/cont

### Processing Multiple Videos:
Deepsea-Detector can also process multiple video files at once. To do this, you'll need to define an **output folder** and a **prefix** for the produced MP4 video files, instead of a single video output.

The CSV data will be merged into a single file.

In [16]:
MODEL_PATH = "deepsea-detector/models/deepsea-detector.pt"
# Include a space between each video you want to process.
VIDEO_INPUT_PATHS = "deepsea-detector/data/test_input_video/coral_trimmed.mp4 deepsea-detector/data/test_input_video/shrimp_trimmed.mp4"
VIDEO_OUTPUT_FOLDER = "example_output"
VIDEO_OUTPUT_PREFIX = "multi_out_"
CSV_OUTPUT_PATH = "multi_out.csv"

# Make an directory called example_output
!mkdir example_output
# Run the script for multiple input videos
!python deepsea-detector/src/detection.py {VIDEO_INPUT_PATHS} \
--detector_path {MODEL_PATH} \
--img_size 640 \
--conf_thres 0.10 \
--period 3 \
--device cuda \
--output_folder {VIDEO_OUTPUT_FOLDER} \
--output_video_prefix {VIDEO_OUTPUT_PREFIX} \
--output_csv {CSV_OUTPUT_PATH}

Traceback (most recent call last):
  File "/content/deepsea-detector/src/detection.py", line 173, in <module>
    model = YOLO(args.detector_path, device=args.device)
  File "/content/deepsea-detector/src/detection.py", line 50, in __init__
    self.model = yolov5.load(model_path, device=device)
  File "/usr/local/lib/python3.10/dist-packages/yolov5/helpers.py", line 35, in load_model
    device = select_device(device)
  File "/usr/local/lib/python3.10/dist-packages/yolov5/utils/torch_utils.py", line 118, in select_device
    assert torch.cuda.is_available() and torch.cuda.device_count() >= len(device.replace(',', '')), \
AssertionError: Invalid CUDA '--device cuda' requested, use '--device cpu' or pass valid CUDA device(s)
